# **Web Scraping for Indeed**

## **Import labraries**

In [ ]:
!pip install bs4

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
import numpy as np
import re

## **Store useful information to DF(Monster)**


In [2]:
#create an empty DF and set pages = 6
monster_table = pd.DataFrame()
page = 6


result = requests.get("https://www.monster.ca/jobs/search/?q=Data-Scientist&intcid=skr_navigation_nhpso_searchMain&stpage=1&page={}".format(page)).text
soup = BeautifulSoup(result, 'lxml')

for jobs in soup.find_all("section", class_ = "card-content"): 
  try:
    job_title = jobs.find('h2', class_ = 'title').text.strip()
  except:
    job_title = None

  try:
    company = jobs.find('div', class_ = 'company').text.strip()
  except:
    company = None

  try:
    location = jobs.find('div', class_ = 'location').text.strip()
  except:
    location = None

  try:
    link = jobs.find('h2', attrs={'class': 'title'}).find('a', href=True)['href']
  except:
    link = None
    
  monster_table = monster_table.append({
      'job_title': job_title, 
      'company': company,
      'location': location,
      'link': link}, ignore_index = True)


In [3]:
#Drop null values
monster_table["job_title"] = monster_table["job_title"].replace("None", np.nan)
monster_table = monster_table.dropna(subset=['job_title'])

#Delect dulicate jobs
monster_table = monster_table.drop_duplicates(['link'], keep='first')

In [4]:
monster_table.head(5)

,company,job_title,link,location
1,Geomodeling,DATA SCIENTIST NOC 2172,https://job-openings.monster.ca/data-scientist...,"Calgary, AB"
3,Procom,Scientifique des données / Data Scientist,https://job-openings.monster.ca/scientifique-d...,"Montréal, QC"
4,Randstad,Data Analyst/Scientist Intermediate -,https://job-openings.monster.ca/data-analyst-s...,"Montreal, QC"
5,Step By Step Professional Services Inc,Data Analytical Specialist/Scientist for Netwo...,https://job-openings.monster.ca/data-analytica...,"Toronto, ON"
6,Apex Systems Inc,Director of Data Engineering,https://job-openings.monster.ca/director-of-da...,"Burnaby, BC"


In [5]:
skills = ["excel", "python", "r", "c","c++","matlab", "sas", "sql", "spss", "tableau", "power bi", "hadoop", "spark", "communication", "presentation", "problem solving", "project management", "consulting", "leadership", "phd", "master", "bachelor"]

In [6]:
for skill in skills:
  monster_table[skill] = np.zeros(len(monster_table))

In [33]:
print(monster_table['link'][3])

https://job-openings.monster.ca/scientifique-des-données-data-scientist-montréal-qc-ca-procom/216645170


In [28]:
for i in range(len(monster_table)):
    detail = requests.get(monster_table.iloc[i,2])
    soup = BeautifulSoup(detail.text, "lxml")
    text = soup.find("div", {'class' : 'details-content'}).text.strip().lower()
    text = tokenizer.tokenize(text)
    text_filter1 = [words for words in text if not words in stop_words]
    text_base1 = [Lemmatizer.lemmatize(words) for words in text_filter1]
    text = ' '.join(words for words in text_base1)
    string = re.sub(r'\,', ' ', text) # remove ","
    string = re.sub('/', ' ', string) # remove "/"
    string = re.sub(r'\(', ' ', string) # remove "("
    string = re.sub(r'\)', ' ', string) # remove ")"
    string = re.sub(' +',' ',string) # remove more than one space
    monster_table.loc[i,'description']=text

KeyError: 0

In [19]:
df=monster_table.iloc[:, 5:]

In [20]:
df=df.fillna(0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.to_csv('webscraping_monster.csv', index=False)

### Clustering

In [ ]:
df_skill=pd.read_csv('webscraping_monster.csv', low_memory=False)

In [ ]:
# Calculating the distance
dist=pdist(df_skill.T, metric='euclidean')
# Calculating the linkage matrix
link = linkage(df_skill.T, method='complete', metric='correlation')
# Draw the dendrogram
plt.figure(figsize=(30,5))
B=dendrogram(link, labels=df_skill.columns)